# Modulation and Coding Performance

SpaceLink includes a registry of modulation and coding combinations. Modes and the
associated error rate performance data is stored in a set of YAML files with a simple
schema such that new modes can be easily added.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

%matplotlib widget

import astropy.units as u
from spacelink.phy.registry import Registry
from spacelink.phy.performance import ErrorMetric

The error rate performance of different modulation and coding options can be compared
in this interactive plot.

In [ ]:
modcod_registry = Registry()
modcod_registry.load()

mode_ids = sorted(modcod_registry.modes.keys())

ebn0 = np.linspace(-2, 10, 100) * u.dB
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12, 8))
fig.subplots_adjust(right=0.55)


@widgets.interact(
    modcod_ids=widgets.SelectMultiple(
        options=mode_ids,
        value=(mode_ids[0],),
        description="Modes:",
        rows=min(12, len(mode_ids)),
        layout=widgets.Layout(width="50%", height="150px"),
        style={"description_width": "auto"},
    )
)
def plot_modcods(modcod_ids):
    ax1.clear()
    ax2.clear()

    colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

    for idx, mode_id in enumerate(modcod_ids):
        color = colors[idx % len(colors)]
        try:
            modcod_ber_perf = modcod_registry.get_performance(mode_id, ErrorMetric.BER)
        except KeyError:
            continue
        ber = modcod_ber_perf.ebno_to_error_rate(ebn0)
        ax1.semilogy(ebn0, ber, label=mode_id, color=color)

        try:
            modcod_wer_perf = modcod_registry.get_performance(mode_id, ErrorMetric.WER)
            wer = modcod_wer_perf.ebno_to_error_rate(ebn0)
            ax2.semilogy(ebn0, wer, label=mode_id, color=color)
        except KeyError:
            pass

    ax1.set_ylabel("Bit Error Rate")
    ax1.grid(True)
    ax2.set_xlabel("$E_b/N_0$ (dB)")
    ax2.set_ylabel("Codeword Error Rate")
    ax2.grid(True)

    if len(modcod_ids) > 0:
        handles, labels = ax1.get_legend_handles_labels()
        ax1.legend(
            handles,
            labels,
            loc="upper left",
            bbox_to_anchor=(1.02, 1.0),
            borderaxespad=0.0,
            title="Modes",
            frameon=True,
        )